# Unit 5 - Financial Planning

In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv()

str

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [3]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [4]:
# Crypto API URLs
# As I am in Australia, I'd prefer to see USD (and not CAD, or EUR, Or GBP, etc.)
# But will request and extract both as USD is default along with the requested curr.
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [6]:
# Fetch current BTC price
btc_data = requests.get(btc_url).json()

# Fetch current ETH price
eth_data = requests.get(eth_url).json()

# Compute current value of my crypto
# Extract the price of BTC in CAD **AND** USD
cad_price_btc = btc_data["data"]["1"]["quotes"]["CAD"]["price"]
usd_price_btc = btc_data["data"]["1"]["quotes"]["USD"]["price"]

# Now do the same for ETH.
cad_price_eth = eth_data["data"]["1027"]["quotes"]["CAD"]["price"]
usd_price_eth = eth_data["data"]["1027"]["quotes"]["USD"]["price"]

# Compute the current monetary value of each "asset"
my_btc_value = my_btc*cad_price_btc
my_btc_val_usd = my_btc*usd_price_btc
#
my_eth_value = my_eth*cad_price_eth
my_eth_val_usd = my_eth*usd_price_eth

# Print current crypto wallet balance
# Add some space with a newline char for better viewing pleasure :)
print(f"\nThe current value of your {my_btc} BTC is ${my_btc_value:0.2f} (US${my_btc_val_usd:0.2f})")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f} (US${my_eth_val_usd:0.2f})")


The current value of your 1.2 BTC is $51803.06 (US$42601.20)
The current value of your 5.3 ETH is $14836.06 (US$12200.71)


### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [8]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [28]:
# Set Alpaca API key and secret
# Fetch my key
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_api_secret = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca API object
# NOTE: According to API documentation,
#       v2 of the API is default
alpaca = tradeapi.REST(alpaca_api_key, alpaca_api_secret)

In [39]:
# Format current date as ISO format
# Set TZ
NYC = 'America/New_York'

# Set start & end
# NOTE: See further notes, where I call the API.
start_date = pd.Timestamp('2015-06-01', tz=NYC).isoformat()
end_date = pd.Timestamp('2021-06-10', tz=NYC).isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to '1D' for Alpaca API
# NOTE: As 'day' is a synonym of '1D', I use it instead.
#timeframe = "1D"
timeframe = "day"

# Get current closing prices for SPY and AGG
# (use a limit=1000 parameter to call the most recent 1000 days of data)
# NOTE:
#   i. As "now" is used if 'end' is omitted, I've not set it
#  ii. As the API returns the most recent data, I've set the
#      start beyond 6 years
investments_data = alpaca.get_barset(tickers, timeframe, limit=1000).df

# Preview DataFrame
investments_data

AGG                                          \
                             open      high       low    close     volume   
time                                                                        
2017-06-21 00:00:00-04:00  109.88  110.0400  109.8800  109.980  1085301.0   
2017-06-22 00:00:00-04:00  110.06  110.0947  109.9500  110.050  1295361.0   
2017-06-23 00:00:00-04:00  110.01  110.1085  109.9800  110.070  1101304.0   
2017-06-26 00:00:00-04:00  110.19  110.2400  110.1300  110.160  1364100.0   
2017-06-27 00:00:00-04:00  109.93  109.9800  109.7901  109.830  2883305.0   
...                           ...       ...       ...      ...        ...   
2021-06-07 00:00:00-04:00  113.97  114.6400  113.9700  114.595  3639918.0   
2021-06-08 00:00:00-04:00  114.84  114.8700  114.7800  114.820  3951702.0   
2021-06-09 00:00:00-04:00  115.43  115.4300  114.9700  115.050  3418086.0   
2021-06-10 00:00:00-04:00  115.30  115.3100  114.8605  115.300  3665934.0   
2021-06-11 00:00:00-04:00  115.31  115.3200  115.1600  115.240  3476373.0   

                              SPY                                       
                             open    high      low   close      volume  
time                                                                    
2017-06-21 00:00:00-04:00     NaN     NaN      NaN     NaN         NaN  
2017-06-22 00:00:00-04:00  242.97  243.53  242.640  242.83  32991335.0  
2017-06-23 00:00:00-04:00  242.91  243.51  242.470  243.16  45792641.0  
2017-06-26 00:00:00-04:00  243.89  244.38  243.045  243.27  43153677.0  
2017-06-27 00:00:00-04:00  243.04  243.38  241.310  241.36  60858189.0  
...                           ...     ...      ...     ...         ...  
2021-06-07 00:00:00-04:00  422.59  422.78  421.190  422.23  41442963.0  
2021-06-08 00:00:00-04:00  423.11  423.21  420.320  422.27  40131441.0  
2021-06-09 00:00:00-04:00  423.18  423.26  421.410  421.62  38368598.0  
2021-06-10 00:00:00-04:00  422.96  424.63  421.550  423.58  43577329.0  
2021-06-11 00:00:00-04:00  424.19  424.42  422.820  424.31  37018077.0  

[1001 rows x 10 columns]

In [ ]:
# Pick AGG and SPY close prices
# YOUR CODE HERE!

# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")

In [ ]:
# Compute the current value of shares
# YOUR CODE HERE!

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

### Savings Health Analysis

In [ ]:
# Set monthly household income
# YOUR CODE HERE!

# Consolidate financial assets data
# YOUR CODE HERE!

# Create savings DataFrame
# YOUR CODE HERE!

# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart
# YOUR CODE HERE!

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
# YOUR CODE HERE!

# Validate saving health
# YOUR CODE HERE!

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2016-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-05-01', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
# (use a limit=1000 parameter to call the most recent 1000 days of data)
# YOUR CODE HERE!

# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Printing the simulation input data
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")